In [1]:
# On importe les bibliothèques utiles pour le projet

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import cv2
# Les functions écrites dans un fichier externe
from functions import *
from collections import namedtuple

from keras_segmentation.models.unet import vgg_unet, resnet50_unet
from keras_segmentation.models.fcn import fcn_8_vgg, fcn_8_resnet50, fcn_32_resnet50
from keras_segmentation.models.segnet import vgg_segnet, resnet50_segnet
from keras import backend as K
from keras.utils import Sequence
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras.utils as image
#from keras.utils import multi_gpu_model
from dilatednet import DilatedNet
from models import vgg16unet, build_vgg16_unet

ModuleNotFoundError: No module named 'tensorflow.compat'

In [ ]:
# Importation du jeu d'entraînement
image_train_dir = 'datas/images/train_clean'
mask_train_dir = 'datas/masks/train_clean'
image_train_list = os.listdir(image_train_dir)
mask_train_list = os.listdir(mask_train_dir)
image_train_list.sort()
mask_train_list.sort()

print(f'. . . . .Number of images: {len(image_train_list)}\n. . . . .Number of masks: {len(mask_train_list)}')

In [ ]:
# Set parameters & hyperparameters
img_height, img_width = 256, 256
img_size = (img_height, img_width)
batch_size = 8
n_classes = 8
samples = 50000
optimizer = 'adam'
loss='categorical_crossentropy'
score_IoU = tf.keras.metrics.MeanIoU(num_classes=8,
                                     name='mean_IoU')
epochs = 10
epochs_optim = 3

In [ ]:
from collections import namedtuple

# Adaptation from the official Cityscapes scripts
# https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py

#--------------------------------------------------------------------------------
# Definitions : 
#--------------------------------------------------------------------------------
# Create tuple subclasses with named fields
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name the 30 classes

    'id'          , # An integer ID that is associated with this label.

    'categoryId'       , # IDs of labels which match to pixel values
                    # corresponding to the 8 main categories.

    'category'    , # The name of the category that this label belongs to
    ] )

#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

labels = [
    #name                             id    catId   category
    Label(  'unlabeled'            ,  0 ,      0 , 'void'),
    Label(  'ego vehicle'          ,  1 ,      0 , 'void'),
    Label(  'rectification border' ,  2 ,      0 , 'void'),
    Label(  'out of roi'           ,  3 ,      0 , 'void'),
    Label(  'static'               ,  4 ,      0 , 'void'),
    Label(  'dynamic'              ,  5 ,      0 , 'void'),
    Label(  'ground'               ,  6 ,      0 , 'void'),
    Label(  'road'                 ,  7 ,      1 , 'flat'),
    Label(  'sidewalk'             ,  8 ,      1 , 'flat'),
    Label(  'parking'              ,  9 ,      1 , 'flat'),
    Label(  'rail track'           , 10 ,      1 , 'flat'),
    Label(  'building'             , 11 ,      2 , 'construction'),
    Label(  'wall'                 , 12 ,      2 , 'construction'),
    Label(  'fence'                , 13 ,      2 , 'construction'),
    Label(  'guard rail'           , 14 ,      2 , 'construction'),
    Label(  'bridge'               , 15 ,      2 , 'construction'),
    Label(  'tunnel'               , 16 ,      2 , 'construction'),
    Label(  'pole'                 , 17 ,      3 , 'object'),
    Label(  'polegroup'            , 18 ,      3 , 'object'),
    Label(  'traffic light'        , 19 ,      3 , 'object'),
    Label(  'traffic sign'         , 20 ,      3 , 'object'),
    Label(  'vegetation'           , 21 ,      4 , 'nature'),
    Label(  'terrain'              , 22 ,      4 , 'nature'),
    Label(  'sky'                  , 23 ,      5 , 'sky'),
    Label(  'person'               , 24 ,      6 , 'human'),
    Label(  'rider'                , 25 ,      6 , 'human'),
    Label(  'car'                  , 26 ,      7 , 'vehicle'),
    Label(  'truck'                , 27 ,      7 , 'vehicle'),
    Label(  'bus'                  , 28 ,      7 , 'vehicle'),
    Label(  'caravan'              , 29 ,      7 , 'vehicle'),
    Label(  'trailer'              , 30 ,      7 , 'vehicle'),
    Label(  'train'                , 31 ,      7 , 'vehicle'),
    Label(  'motorcycle'           , 32 ,      7 , 'vehicle'),
    Label(  'bicycle'              , 33 ,      7 , 'vehicle'),
    Label(  'license plate'        , -1 ,      7 , 'vehicle')]

# Function to map labels and categories
def map_id2category(labels):
    """ This function maps the ~30 labels' IDs
        to the 8 main categories' IDs
    """
    cat_label = {label.id: label.categoryId for label in labels}
    
    # Get the mapping
    map_category = np.vectorize(cat_label.get)
    return map_category

In [ ]:
class DataGenerator(Sequence):
    """Generate data for Keras
    """

    # ==== Initialize the class ==== #
    def __init__(self, input_path, target_path, img_size, batch_size, 
                 labels, n_classes, shuffle):
        self.input_path = input_path
        self.target_path = target_path
        self.targetSize = img_size
        self.batch_size = batch_size
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()


    # ==== Set the number of batches per epoch ==== #
    def __len__(self):
        return int(np.floor(len(self.input_path) / float(self.batch_size)))
    
    
    # ==== Call a method at the end of each epoch ==== #
    def on_epoch_end(self):
        # [0,1,2,3,4... nb_image]
        self.indexes = np.arange(len(self.input_path))
        
        # [2,4,1,3,0... nb_image]
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    
    # ==== Get batches of images & masks ==== #
    def __getitem__(self, index):
       # Genere batch_size nombre d'ID de row de DATA (batch_size=2, [0,1])
       currentBatchIdsRow = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
    
       batch_x, batch_y = [], []
    
       # Iterate over indexes
       for i, rowId in enumerate(currentBatchIdsRow):
           # Load image, set target size, convert to array, normalize
           image = img_to_array(
               load_img(f'{self.input_path[i]}', 
                        target_size=img_size))/255.
           batch_x.append(image)

           # Load original mask, set target size and convert to array
           mask30 = img_to_array(
               load_img(f'{self.target_path[i]}', color_mode = "grayscale", 
                        target_size=img_size))
           
           # Map the 30 classes to 8 main categories
           map_category = map_id2category(labels)
           mask = map_category(mask30)
           mask = to_categorical(mask, self.n_classes)                   
           batch_y.append(mask)
     
       return np.array(batch_x), np.array(batch_y)

In [ ]:
# Generate data
trainGen = DataGenerator(train_input_path, train_mask_path, img_size, 
                         batch_size, labels, n_classes, shuffle=False)
valGen = DataGenerator(val_input_path, val_mask_path, img_size, 
                       batch_size, labels, n_classes, shuffle=False)

# Check data shape
X_train, y_train = trainGen.__getitem__(0)
print('Train shapes (X and y): ', X_train.shape, y_train.shape)
X_val, y_val = valGen.__getitem__(0)
print('Val shapes (X and y): ', X_val.shape, y_val.shape)